https://huggingface.co/NLP-LTU/bertweet-large-sexism-detector?

In [17]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer,pipeline
import torch

In [18]:
model = AutoModelForSequenceClassification.from_pretrained('NLP-LTU/bertweet-large-sexism-detector')
tokenizer = AutoTokenizer.from_pretrained('NLP-LTU/bertweet-large-sexism-detector') 
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)
prediction=classifier("")

In [19]:
print(prediction)

[{'label': 'not sexist', 'score': 0.9985785484313965}]
